## Builder‘s Guide

### Layers and Modules

- Mulitple layers are combined into modules

In [8]:
import torch
from torch import nn
from torch.nn import functional as F

In [9]:
net = nn.Sequential(nn.LazyLinear(256), nn.ReLU(), nn.LazyLinear(10))

X = torch.rand((2, 10))
net(X).shape

torch.Size([2, 10])

#### A Custom Module

In [10]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.LazyLinear(256)
        self.out = nn.LazyLinear(10)
    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))
net = MLP()
net(X).shape

torch.Size([2, 10])

#### The Sequential Module

- A method to append modules one by one to a list
- A forward propagation method to pass an input through the chain of modules, in the same order as they were appended

In [198]:
# using "childrne" class to access added modules

class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, moduels in enumerate(args):
            self.add_module('l' + str(idx), moduels)
    def forward(self, X):
        for module in self.children():
            X = module(X)
        return X   

In [199]:
net = MySequential(nn.LazyLinear(256),nn.LazyLinear(256), nn.ReLU(), nn.LazyLinear(10))
net(X).shape

C:\Users\shaohan.tian\scoop\apps\miniconda3\current\envs\d2l\lib\site-packages\torch\nn\modules\lazy.py:178: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


torch.Size([2, 10])

In [200]:
net

MySequential(
  (l0): Linear(in_features=20, out_features=256, bias=True)
  (l1): Linear(in_features=256, out_features=256, bias=True)
  (l2): ReLU()
  (l3): Linear(in_features=256, out_features=10, bias=True)
)

In [201]:
net.l0

Linear(in_features=20, out_features=256, bias=True)

#### Executing Code in the Forward Propagation Method

- Perform arbitrary matchmatical operations, not simpely relying on predifned NNs
- sharing one linear layer and add some mathmatical operaitons

In [45]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        # Random weight parameters that will not compute gradients and
        # therefore keep constant during training
        self.rand_weight = torch.rand((20, 20))
        self.linear = nn.LazyLinear(20)
    def forward(self, X):
        X = self.linear(X)
        X = F.relu(X @ self.rand_weight + 1)
        # Reuse the fully connected layer. This is equivalent to sharing
        # parameters with two fully connected layers
        X = self.linear(X)
        # Control flow
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

In [81]:
X = torch.arange(40.0).reshape(2, 20)
net = FixedHiddenMLP()
net(X)

C:\Users\shaohan.tian\scoop\apps\miniconda3\current\envs\d2l\lib\site-packages\torch\nn\modules\lazy.py:178: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


tensor(0.2432, grad_fn=<SumBackward0>)

In [83]:
# mixing various modules together
class NestMlp(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.LazyLinear(20), nn.ReLU(),
                                nn.LazyLinear(50), nn.ReLU())
        self.linear = nn.LazyLinear(60)
    def forward(self, X):
        return self.linear(self.net(X))
    
chimera = nn.Sequential(NestMlp(), nn.LazyLinear(20), FixedHiddenMLP())
chimera(X)

tensor(0.0621, grad_fn=<SumBackward0>)

In [202]:
chimera

Sequential(
  (0): NestMlp(
    (net): Sequential(
      (0): Linear(in_features=20, out_features=20, bias=True)
      (1): ReLU()
      (2): Linear(in_features=20, out_features=50, bias=True)
      (3): ReLU()
    )
    (linear): Linear(in_features=50, out_features=60, bias=True)
  )
  (1): Linear(in_features=60, out_features=20, bias=True)
  (2): FixedHiddenMLP(
    (linear): Linear(in_features=20, out_features=20, bias=True)
  )
)

#### Summary

- Layers are modules. Many layers can comprise a module. Many modules can comprise a module.  
- A module can contain code. Modules take care of lots of housekeeping, including parameter initialization and backpropagation. Sequential concatenations of layers and modules are handled by the Sequential module

#### Exerises

1. What kinds of problems will occur if you change MySequential to store modules in a Python list?

In [260]:
class MySequentialExe(nn.Module):
    def __init__(self, *args):
        super().__init__()

        self.layers = []
        for idx, layer in enumerate(args): # convert module inputs to list
            self.layers.append(layer)

    def forward(self, X):
        for block in self.layers:
            X = block(X)
        return X

x = torch.rand(size=(2, 10))
net = MySequentialExe(nn.LazyLinear(8), nn.LazyLinear(4), nn.LazyLinear(1))
net

C:\Users\shaohan.tian\scoop\apps\miniconda3\current\envs\d2l\lib\site-packages\torch\nn\modules\lazy.py:178: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


MySequentialExe()

In [261]:
net(x)

tensor([[-0.4829],
        [-0.4826]], grad_fn=<AddmmBackward0>)

In [262]:
net.state_dict()

OrderedDict()

2. Implement a module that takes two modules as an argument, say net1 and net2 and returns the concatenated output of both networks in the forward propagation. This is also called a **parallel module**.

3. Assume that you want to concatenate multiple instances of the same network. Implement a factory function that generates multiple instances of the same module and build a larger network from it.

### Parameter Management

- Accessing parameters for debugging, diagnostic, and visualizations
- Sharing parameters across different model components.

In [203]:
import torch
from torch import nn

In [204]:
net = nn.Sequential(nn.LazyLinear(8),
                    nn.ReLU(),
                    nn.LazyLinear(1))
X = torch.rand(size=(2, 4))
net(X)

C:\Users\shaohan.tian\scoop\apps\miniconda3\current\envs\d2l\lib\site-packages\torch\nn\modules\lazy.py:178: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


tensor([[0.3653],
        [0.3266]], grad_fn=<AddmmBackward0>)

In [205]:
net

Sequential(
  (0): Linear(in_features=4, out_features=8, bias=True)
  (1): ReLU()
  (2): Linear(in_features=8, out_features=1, bias=True)
)

#### Parameter access

In [206]:
net[2].state_dict()

OrderedDict([('weight',
              tensor([[-0.1951,  0.1670,  0.3505,  0.1089, -0.0129,  0.2969,  0.2359, -0.2969]])),
             ('bias', tensor([0.2740]))])

In [211]:
type(net[2].bias), net[2].bias.data

(torch.nn.parameter.Parameter, tensor([0.2740]))

In [214]:
# we have not invoked backprogation for this NN
net[2].weight.grad == None

True

In [217]:
# all parameters
[(name, param.shape) for name, param in net.named_parameters()]

[('0.weight', torch.Size([8, 4])),
 ('0.bias', torch.Size([8])),
 ('2.weight', torch.Size([1, 8])),
 ('2.bias', torch.Size([1]))]

#### Tied parameters

In [220]:
# we need to give a shared layer a name so that we can refer to
# its  parameters
shared = nn.LazyLinear(8)
net = nn.Sequential(nn.LazyLinear(8), nn.ReLU(),
                    shared, nn.ReLU(),
                    shared, nn.ReLU(),
                    nn.LazyLinear(1))
net(X)

C:\Users\shaohan.tian\scoop\apps\miniconda3\current\envs\d2l\lib\site-packages\torch\nn\modules\lazy.py:178: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


tensor([[0.1073],
        [0.1005]], grad_fn=<AddmmBackward0>)

In [221]:
net

Sequential(
  (0): Linear(in_features=4, out_features=8, bias=True)
  (1): ReLU()
  (2): Linear(in_features=8, out_features=8, bias=True)
  (3): ReLU()
  (4): Linear(in_features=8, out_features=8, bias=True)
  (5): ReLU()
  (6): Linear(in_features=8, out_features=1, bias=True)
)

In [222]:
net[2].weight.data == net[4].weight.data

tensor([[True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True]])

In [226]:
# modify one of the parameters and keep still
net[2].weight.data[0, 0] = 100
net[2].weight.data == net[4].weight.data

tensor([[True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True]])